In [ ]:
from route import Route
route = Route(choose_specific=True)

# route.insert_to_control_stops(choose_specific=True)
# route.insert_to_camping(choose_specific=False)

In [ ]:
from gps import GPS

gps = GPS('COM8', route=route, choose_specific=True)

In [ ]:
current_position = gps.get_current_location()
print(f"{current_position['latitude']}, {current_position['longitude']}, {current_position['cumDistance']}")
gps.save_data2folder()

In [ ]:
# from db_querier import DbQuerier

# db_querier = DbQuerier(gps)

In [ ]:
# v = db_querier.query_velocity()
# soc = db_querier.query_soc()

# db_querier.save_data2folder()

In [ ]:
from optimal_reader import OptimalReader

optimal_reader = OptimalReader(choose_specific=True)
DP_data = optimal_reader.get_optimal_data

In [ ]:
from route_api import RouteAPI

current_position = {'longitude': 133.8796310,
                    'latitude':-23.6950798}
# current_position = {'longitude': 130.842666,
#                     'latitude':-12.465516}
final_position = {'longitude': 133.8750197,
                  'latitude': -23.7090616}
delta_spacing = 5000.0 # in meters
number_sites = 150

api_route = RouteAPI(route)

# api_route_df = api_route.cut_route_data(current_position, number_sites=number_sites, print_is_requested=True)
api_route_df = api_route.cut_route_data(current_position, number_sites=number_sites, delta_spacing=delta_spacing, print_is_requested=True)
# api_route_df = api_route.cut_route_data(current_position=current_position, final_position=final_position, number_sites=number_sites, print_is_requested=False)
# print(route.find_closest_row(current_position))

print(route.get_route_data.head())

In [ ]:
from preprocessor import Preprocessor
preprocessor = Preprocessor(choose_specific=True)

In [ ]:
import pandas as pd
from Solcast_executer import SolcastExecuter
solcast_executer = SolcastExecuter()

# current_position = {'longitude': 130.9064683,
#                     'latitude': -12.4479361}
# raw_df = solcast_executer.get_forecast(current_position)
raw_SC_df = solcast_executer.get_forecasts(api_route_df, hours_in_advance=48)
print(raw_SC_df)

sites_df = api_route_df.copy()
hours_in_advance = None
forecast_df, preprocessed_df = preprocessor.preprocess(api_route_df, sites_df, raw_SC_df, hours_in_advance)
preprocessor.save_data2folder()
print(preprocessed_df)

In [ ]:
from Meteotest_executer import MeteotestExecuter
from Meteotest_requester import MeteotestRequester
from Meteotest_parser import MeteotestParser

api_parser = MeteotestParser()
api_requester = MeteotestRequester(parser=api_parser, print_is_requested=False)
api_executer = MeteotestExecuter(requester=api_requester)

In [ ]:
# api_requester.get_site_info(print_is_requested=True)
# api_executer.get_all_site_id

In [ ]:
# api_requester.get_site_add(name="0", position=current_position, print_is_requested=True)
# api_executer.add_sites(api_route_df, print_is_requested=True)

In [ ]:
# api_requester.get_site_delete(585394, print_is_requested=True)
# api_executer.delete_all_sites()

In [ ]:
# site_id = 54689
# name = "404"
# position = {'latitude': 136,
#             'longitude': -30}
# api_requester.get_site_edit(site_id, name=name, position=position)

# api_route_df.index=api_executer.get_all_site_id ########TODO ADAPT DIRECTLY 
# api_executer.edit_sites(api_route_df)

In [ ]:
raw_SF_df = api_requester.get_solar_forecast()
print(raw_SF_df.head())

# raw_CM_df = api_requester.get_solar_forecast_cloudmove()
# print(raw_CM_df)

# raw_SF_df, _, raw_CM_df, _ = api_executer.get_new_forecasts()

# print(raw_SF_df.head(), raw_CM_df.head())

sites_df = api_requester.get_current_sites
hours_in_advance = None

# SF
forecast_df, preprocessed_df = preprocessor.preprocess(api_route_df, sites_df, raw_SF_df, hours_in_advance)
preprocessor.save_data2folder()
print(preprocessed_df)

# # CM
# forecast_df, preprocessed_df = preprocessor.preprocess(api_route_df, sites_df, raw_CM_df, hours_in_advance)
# preprocessor.save_data2folder()
# print(preprocessed_df)

In [ ]:
from time_space_forecaster import TimeSpaceForecaster
time_space_forecaster = TimeSpaceForecaster(route, gps=None, db_querier=None, optimal_reader=None)

current_position = {'longitude': 133.1573891,
                    'latitude':-15.3479029}

current_cumDistance = route.find_closest_row(current_position)[0]['cumDistance']

current_position = {**current_position,
                    'cumDistance': current_cumDistance,
                    'cumDistance_km': current_cumDistance/1000}

# time_space_forecaster.get_cum_distance_forecast(current_position, type="", time={'hour': 17, 'minute': 0}, speed=60)
# time_space_forecaster.get_time_at_next_control_stop(current_position, type="max_speed", speed=60)

In [ ]:
#BUG PAN, ZOOM -> XAXIS

import pandas as pd
import dash
import constants
from dash import Dash, html, dcc, dash_table
from dash.dependencies import Input, Output, State
from functions import rad2deg

import random

row, _ = route.find_closest_row_cumDistance(1489000)
current_position = {'longitude': row['longitude'],
                    'latitude': row['latitude'],
                    'cumDistance': row['cumDistance'],
                    'cumDistance_km': row['cumDistance'] / 1000}


control_stops_df = route.get_control_stops_data
sites_df = api_requester.get_current_sites
current_position['cumDistance'] = route.find_closest_row(current_position, print_is_requested=False)[0]['cumDistance']
current_position['cumDistance_km'] = current_position['cumDistance'] / 1000

# Get the current time
now = pd.Timestamp.now(tz=constants.TIMEZONE)
current_hour = now.hour
current_minute = now.minute
# Create hour and minute options based on constraints
hour_options = [{'label': str(i), 'value': i} for i in range(current_hour, 18)]
minute_options = [{'label': str(i), 'value': i} for i in range(current_minute, 60)] if current_hour == now.hour else [{'label': str(i), 'value': i} for i in range(60)]

# aCe images
url_image = 'https://avatars.githubusercontent.com/u/115261983?s=280&v=4'

app = Dash(__name__)
app.title = "Strategy Interface"

## Time
# Slider
height_slider = 1000
unique_times = preprocessed_df.index.get_level_values('time').unique()
slider_marks = {i: time.strftime('%H:%M') for i, time in enumerate(unique_times)}

# Dropdown
unique_days = unique_times.map(lambda x: x.strftime('%d %B %Y')).unique().to_list()

# Update time
update_sec = 40

## Camping
camping_df = route.get_camping_data
camping_cut_df = camping_df[['name','latitude','longitude','cumDistance']]
camping_cut_df.sort_values(by='cumDistance', ascending=True, inplace=True)

## Map
route_df = route.get_route_data
max_cumDistance_km = route_df['cumDistance'].max() / 1000
row_current_cumDistance, _ = route.find_closest_row(current_position)
min_lng = route_df['longitude'].min()
min_lat = route_df['latitude'].min()
max_lng = route_df['longitude'].max()
max_lat = route_df['latitude'].max()
middle_lat = (max_lat + min_lat) / 2
middle_lng = (max_lng + min_lng) / 2

## Plot
x_cumDistance_km = route_df['cumDistance'] / 1000
x_cumDistance_str = 'Cumulative Distance (km)'

grid_color = 'lightgray'
box_color = 'black'

height_plot = 280
width_padding = '10px'

# Control stops, start, and finish lines
control_stops_df = route.get_control_stops_data
control_stops = control_stops_df.iloc[1:-1]
cs_vertical_lines = []
for _, row in control_stops.iterrows():
    x_value = row['cumDistance'] / 1000
    cs_vertical_lines.append(
        dict(type='line',
             x0=x_value,
             x1=x_value,
             y0=0,
             y1=1,
             yref='paper',
             line=dict(color='orange', dash='dash', width=1)
        ) #, hovertext=row['town'] + ': ' + row['location'], hoverinfo='text'
    )

start_finish = control_stops_df.iloc[[0, -1]]
start_finish_lines = []
for _, row in start_finish.iterrows():
    x_value = row['cumDistance'] / 1000
    start_finish_lines.append(
        dict(type='line',
             x0=x_value,
             x1=x_value,
             y0=0,
             y1=1,
             yref='paper',
             line=dict(color='black', width=0.5)
        ) #, hovertext=row['town'] + ': ' + row['location'], hoverinfo='text'
    )

lenght_cs = len(cs_vertical_lines + start_finish_lines)

current_position_line = [{
            'type': 'line',
            'x0': current_position['cumDistance_km'],
            'x1': current_position['cumDistance_km'],
            'y0': 0,
            'y1': 1,
            'yref': 'paper',
            'line': {
                'color': 'red',
                'width': 1}
            },
        ]

# Initialize the vertical lines for the shapes
vertical_lines = cs_vertical_lines + start_finish_lines + current_position_line

## Function to initialize plots
def initialize_plot(title:str, xs:list=[], ys:list=[], names:list=[]) -> dict:
    # Handle empty lists
    if not xs or not ys:
        traces = []
    
    else:
        if len(xs) != len(ys):
            raise ValueError("The number of x and y lists must be the same.")
            
        # Ensure all elements in xs and ys are pandas Series
        if not all(isinstance(x, pd.Series) for x in xs) or not all(isinstance(y, pd.Series) for y in ys):
            raise ValueError("All elements of xs and ys must be pandas Series.")

        traces = [{
            'x': x,
            'y': y,
            'type': 'scatter',
            'mode': 'lines',
            'name': name if names else '',
            'hoverinfo': 'none'
        } for x, y, name in zip(xs, ys, names)]
    
    return {
        'data': traces,
        'layout': {
            'margin': {"r": 0, "t": 0},
            'xaxis': {
                'title': x_cumDistance_str,
                'showgrid': True,
                'gridcolor': grid_color,
                'showline': True,
                'linecolor': box_color},
            'yaxis': {
                'title': title,
                'showgrid': True,
                'gridcolor': grid_color,
                'showline': True,
                'linecolor': box_color},
            'height': height_plot,
            'legend': {
                'x': 1,
                'y': 1,
                'xanchor': 'right',
                'yanchor': 'top'
            },
            'shapes': vertical_lines
        }
    }


app.layout = html.Div([
    dcc.Interval(id='interval_update', interval=update_sec*1000, n_intervals=0),
    dcc.Interval(id='interval_update_ds_time', interval=10*1000, n_intervals=0),
    dcc.Store(id='store_current_position', data=current_position),
    dcc.Store(id='store_driver_stop', data={}),

    # Container for the scouts
    html.Div([
        # Current position
        html.Div([
            html.Table([
                # Header with merged cells
                html.Tr([html.Th('Current position', colSpan=2)]),
                
                # Remaining rows
                html.Tr([html.Td('Latitude'), html.Td('Longitude')]),
                html.Tr([html.Td(id='current_lat'), html.Td(id='current_lng')]),
                html.Tr([html.Td('Distance from Darwin'), html.Td('Distance to Adelaide')]),
                html.Tr([html.Td(id='dist_darw'), html.Td(id='dist_ade')]),
            ]),
        ], style={'padding-left': width_padding, 'padding-right': width_padding}),

        # Control stop
        html.Div([
            html.Table([
                # Header with merged cells
                html.Tr([html.Th('Next control stop', colSpan=2)]),
                
                # Remaining rows
                html.Tr([html.Th(id='cs_name', colSpan=2)]),
                html.Tr([html.Td('Latitude'), html.Td('Longitude')]),
                html.Tr([html.Td(id='cs_lat'), html.Td(id='cs_lng')]),
                html.Tr([html.Td('Distance from Darwin'), html.Td('Distance from us')]),
                html.Tr([html.Td(id='cs_dist_dar'), html.Td(id='cs_dist_us')]),
            ]),

        html.Table([
            html.Tr([html.Th('At max speed', colSpan=2)]),
            html.Tr([html.Td('Time'), html.Td(id='cs_max_v_time')]),
            html.Tr([html.Th('At optimal speed', colSpan=2)]),
            html.Tr([html.Td('Time'), html.Td(id='cs_opt_v_time')]),
            html.Tr([html.Th('At average speed', colSpan=2)]),
            html.Tr([html.Td('Time'), html.Td(id='cs_mean_v_time')]),
        ]),

        html.Br(),
        html.Div([
            dcc.Input(id='cs_input_v_time', type='number', min=50, max=130, placeholder='Velocity km/h'),
            html.Button('Submit', id='submit_cs_v_time', disabled=True)
        ]),

        html.Br(),
        html.Table([
            html.Tr([html.Th('At input speed', colSpan=2)]),
            html.Tr([html.Td('Time'), html.Td(id='cs_output_velocity')]),
        ]),
        ], style={'display': 'flex', 'flex-direction': 'column', 'align-items': 'center', 'padding-left': width_padding, 'padding-right': width_padding}),

        # Driver stop
        html.Div([
            html.Table([
                # Header with merged cells
                html.Tr([html.Th('Next driver stop', colSpan=2)]),
                
                # Remaining rows
                html.Tr([html.Th('At max speed', colSpan=2)]),
                html.Tr([html.Td('Latitude'), html.Td('Longitude')]),
                html.Tr([html.Td(id='ds_max_v_lat'), html.Td(id='ds_max_v_lng')]),
                html.Tr([html.Td('Distance from Darwin'), html.Td('Distance from us')]),
                html.Tr([html.Td(id='ds_max_v_dist_darw'), html.Td(id='ds_max_v_dist_us')]),

                html.Tr([html.Th('At optimal speed', colSpan=2)]),
                html.Tr([html.Td('Latitude'), html.Td('Longitude')]),
                html.Tr([html.Td(id='ds_opt_v_lat'), html.Td(id='ds_opt_v_lng')]),
                html.Tr([html.Td('Distance from Darwin'), html.Td('Distance from us')]),
                html.Tr([html.Td(id='ds_opt_v_dist_darw'), html.Td(id='ds_opt_v_dist_us')]),

                html.Tr([html.Th('At average speed', colSpan=2)]),
                html.Tr([html.Td('Latitude'), html.Td('Longitude')]),
                html.Tr([html.Td(id='ds_mean_v_lat'), html.Td(id='ds_mean_v_lng')]),
                html.Tr([html.Td('Distance from Darwin'), html.Td('Distance from us')]),
                html.Tr([html.Td(id='ds_mean_v_dist_darw'), html.Td(id='ds_mean_v_dist_us')]),
            ]),

            html.Div([
                html.Div([
                    dcc.Dropdown(
                        id='dropdown_ds_hour',
                        options=hour_options,
                        value=current_hour,
                        clearable=False
                    ),
                    dcc.Dropdown(
                        id='dropdown_ds_minute',
                        options=minute_options,
                        value=current_minute,
                        clearable=False
                    ),
                ], style={'display': 'flex', 'flex-direction': 'row', 'align-items': 'center'}
                ),
                html.Button('Submit', id='submit_time_ds', disabled=True),
                html.Div(id='output_time')
            ], style={'display': 'flex', 'flex-direction': 'column', 'align-items': 'center'}),
        ], style={'padding-left': width_padding, 'padding-right': width_padding}),

        # Overnight stop
        html.Div([
            html.Table([
                # Header with merged cells
                html.Tr([html.Th('Next overnight stop', colSpan=2)]),
                
                # Remaining rows
                html.Tr([html.Th('At max speed', colSpan=2)]),
                html.Tr([html.Td('Latitude'), html.Td('Longitude')]),
                html.Tr([html.Td(id='os_max_v_lat'), html.Td(id='os_max_v_lng')]),
                html.Tr([html.Td('Distance from Darwin'), html.Td('Distance from us')]),
                html.Tr([html.Td(id='os_max_v_dist_darw'), html.Td(id='os_max_v_dist_us')]),

                html.Tr([html.Th('At optimal speed', colSpan=2)]),
                html.Tr([html.Td('Latitude'), html.Td('Longitude')]),
                html.Tr([html.Td(id='os_opt_v_lat'), html.Td(id='os_opt_v_lng')]),
                html.Tr([html.Td('Distance from Darwin'), html.Td('Distance from us')]),
                html.Tr([html.Td(id='os_opt_v_dist_darw'), html.Td(id='os_opt_v_dist_us')]),

                html.Tr([html.Th('At average speed', colSpan=2)]),
                html.Tr([html.Td('Latitude'), html.Td('Longitude')]),
                html.Tr([html.Td(id='os_mean_v_lat'), html.Td(id='os_mean_v_lng')]),
                html.Tr([html.Td('Distance from Darwin'), html.Td('Distance from us')]),
                html.Tr([html.Td(id='os_mean_v_dist_darw'), html.Td(id='os_mean_v_dist_us')]),
            ])
        ], style={'padding-left': width_padding, 'padding-right': width_padding}),

        # Possible overnight stop
        html.Div([
            dash_table.DataTable(id='os_table',
                columns=[
                    {'name': 'Name', 'id': 'name'},
                    {'name': 'Latitude', 'id': 'latitude'},
                    {'name': 'Longitude', 'id': 'longitude'},
                    {'name': 'Distance from Darwin (km)', 'id': 'cumDistance_km'}
                ],
                style_cell_conditional=[
                    {'if': {'column_id': 'name'}, 'textAlign': 'left'},
                    {'if': {'column_id': 'latitude'}, 'textAlign': 'center'},
                    {'if': {'column_id': 'longitude'}, 'textAlign': 'center'},
                    {'if': {'column_id': 'cumDistance_km'}, 'textAlign': 'right'},
                ]
            ),

            html.Br(),
        
            html.Div([
                dcc.Input(id='input_cum_dist', type='number', min=current_position['cumDistance_km'], max=max_cumDistance_km, placeholder='Distance from Darwin km'),
                html.Button('Submit', id='submit_cum_dist', disabled=True)
            ]),

            html.Div([
                html.Div([
                    dcc.Input(id='input_lat', type='number', min=constants.GEO['latitude']['min'], max=constants.GEO['latitude']['max'], placeholder='Latitude in °'),
                    dcc.Input(id='input_lng', type='number', min=constants.GEO['longitude']['min'], max=constants.GEO['longitude']['max'], placeholder='Longitude in °'),
                ], style={'display': 'flex', 'flex-direction': 'column', 'align-items': 'center'}
                ),
                html.Button('Submit', id='submit_lat_lng', disabled=True),
                html.Div(id='output_lat_lng')
            ], style={'display': 'flex', 'flex-direction': 'row', 'align-items': 'center'}),

            html.Table([
                # Remaining rows
                html.Tr([html.Th('At 17:00 at constant velocity', colSpan=2)]),
                html.Tr([html.Td(id='os_output_velocity'), html.Td('km/h')]),
                html.Tr([html.Td('Latitude'), html.Td('Longitude')]),
                html.Tr([html.Td(id='os_output_lat'), html.Td(id='os_output_lng')]),
                html.Tr([html.Td('Distance from Darwin'), html.Td('Distance from us')]),
                html.Tr([html.Td(id='os_output_dist_darw'), html.Td(id='os_output_dist_us')]),
            ]),
        ], style={'display': 'flex', 'flex-direction': 'column', 'alignItems': 'center', 'padding-left': width_padding, 'padding-right': width_padding}),
    ], style={'display': 'flex', 'alignItems': 'center', 'padding-left': '20px', 'padding-right': '20px'}),


    # Container for the columns
    html.Div([
        # Map and static plots
        html.Div(children=[
            # Map
            dcc.Graph(id='map',
                config={'displayModeBar': 'hover'},
                figure={
                    'data': [{
                        'type': 'scattermapbox',
                        'lat': route_df['latitude'],
                        'lon': route_df['longitude'],
                        'mode': 'lines+markers',
                        'marker': {'size': 3},
                        'line': {'color': 'blue'},
                        'hoverinfo': 'none',
                        'name': 'Route'
                    },
                    {
                        'type': 'scattermapbox',
                        'lat': api_route_df['latitude'],
                        'lon': api_route_df['longitude'],
                        'mode': 'markers',
                        'marker': {'size': 3},
                        'line': {'color': 'white'},
                        'hoverinfo': 'none',
                        'name': 'Weather locations'
                    },
                    {
                        'type': 'scattermapbox',
                        'lat': control_stops_df['latitude'],
                        'lon': control_stops_df['longitude'],
                        'mode': 'markers',
                        'marker': {'size': 5},
                        'line': {'color': 'orange'},
                        'hovertext': (control_stops_df['town'] + ": " + control_stops_df['location']).to_list(),
                        'hoverinfo': 'text',
                        'name': 'Control Stops'
                    },
                    {
                        'type': 'scattermapbox',
                        'lat': [current_position['latitude']],
                        'lon': [current_position['longitude']],
                        'mode': 'markers',
                        'marker': {'size': 10, 'color': 'red'},
                        'name': 'Aletch',
                        'hovertext': 'Current position',
                        'hoverinfo': 'text'
                    },
                    {
                        'type': 'scattermapbox',
                        'lat': [],
                        'lon': [],
                        'mode': 'markers',
                        'marker': {'size': 10, 'color': 'yellow'},
                        'name': 'Next overnight stop',
                        'hovertext': 'Next overnight stop',
                        'hoverinfo': 'text'
                    }],
                    'layout': {
                        'mapbox': {
                            'style': "mapbox://styles/mapbox/satellite-streets-v11",
                            'accesstoken': constants.TOKEN_MAPBOX,
                            'center': {'lat': middle_lat, 'lon': middle_lng},
                            'zoom': 3.5,
                            'minzoom': 3.5
                        },
                        'margin': {'l': 0, 'r': 0, 'b': 0, 't': 0},
                        'legend': {
                            'x': 0,
                            'y': 1,
                            'xanchor': 'left',
                            'yanchor': 'top',
                            'bgcolor': 'transparent',
                            'font': {'color': 'white'}
                        }
                    }
                }
            ),

            # Velocity
            html.Br(),
            dcc.Graph(id='plot_velocity',
                figure=initialize_plot(title='Velocity (km/h)', xs=[x_cumDistance_km, DP_data['cumDistance'] / 1000], ys=[route_df['maxSpeed'], DP_data['velocity']], names=['Max speed', 'Optimal DP']), #
            ),

            # SoC
            html.Br(),
            dcc.Graph(id='plot_soc',
                figure=initialize_plot(title='SoC (%)', xs=[DP_data['cumDistance'] / 1000], ys=[DP_data['soc']], names=['Optimal DP']),
            ),

            # Altitude
            html.Br(),
            dcc.Graph(id='plot_altitude',
                figure=initialize_plot(title='Altitude (m)', xs=[x_cumDistance_km], ys=[route_df['altitudeSmoothed']], names=['Smoothed']),
            ),

            # Inclination
            html.Br(),
            dcc.Graph(id='plot_inclination',
                figure=initialize_plot(title='Inclination (°)', xs=[x_cumDistance_km], ys=[route_df['inclinationSmoothed'].apply(rad2deg)], names=['Smoothed']),
            )
        ], style={'padding': width_padding, 'flex': 1}),

        # Time slider
        html.Div(
            dcc.RangeSlider(
                id='rangeSlider_hour_minute',
                min=0,
                max=len(unique_days) - 1,
                value=[0, len(unique_days) - 1],
                marks=slider_marks,
                vertical=True,
                step=1,
                verticalHeight=height_slider
            ),
            style={'width': '70px', 'height':height_slider},
            id='rangeSlider_hour_minute-container'
        ),

        # Info and Weather plots
        html.Div(children=[
            html.Div([
                # Dropdown
                dcc.Dropdown(options=[{'label': day, 'value': day} for day in unique_days],
                            value=unique_days[0],
                            id='dropdown_day_month_year',
                            style={'width': '200px', 'margin-right': '20px'}),
                
                # Last update
                html.Div([
                    html.Div(id='last_update_sf',
                            style={'color': 'gray'}),
                    html.Div(id='last-update-cm',
                            style={'color': 'gray'}),
                    html.Div(id='last_update_sc',
                            style={'color': 'gray'}),
                ], style={'display': 'flex', 'flexDirection': 'column', 'margin-right': '20px'}),

                # Image
                html.Img(src=url_image,
                        style={'height': '100px', 'width': 'auto', 'margin-left': '20px'}),
            ], style={'display': 'flex', 'alignItems': 'center', 'padding-left': '20px', 'padding-right': '20px'}),

            dcc.Graph(id='plot_globalIrradiance',
                figure=initialize_plot(title='Global irradiance (W/m²)')),
            dcc.Graph(id='plot_temperature',
                figure=initialize_plot(title='Temperature (°)')),
            dcc.Graph(id='plot_wind',
                figure=initialize_plot(title='Wind (km/h)')),
            dcc.Graph(id='plot_precipitation',
                figure=initialize_plot(title='Precipitation (mm)')),

        ], style={'padding': width_padding, 'flex': 1})
    ], style={'display': 'flex', 'flex-direction': 'row'})
], style={'display': 'flex', 'flex-direction': 'column'})

### Callbacks
# Last update forecasts
@app.callback(
    Output('last_update_sf', 'children'),
    Output('last-update-cm', 'children'),
    Output('last_update_sc', 'children'),
    Input('interval_update', 'n_intervals')
)
def update_output_div(n):
    time_SF_str = api_requester.get_solar_forecast_last_time
    if time_SF_str is pd.NaT:
        print_SF = "Last update SF: No data available"
    else:
        print_SF = f"Last update SF: {time_SF_str.strftime('%d %B %Y at %H:%M')}"

    time_CM_str = api_requester.get_solar_forecast_cloudmove_last_time
    if time_CM_str is pd.NaT:
        print_CM = "Last update CM: No data available"
    else:
        print_CM = f"Last update CM: {time_CM_str.strftime('%d %B %Y at %H:%M')}"

    time_SC_str = solcast_executer.get_solcast_last_time
    if time_SC_str is pd.NaT:
        print_SC = "Last update SC: No data available"
    else:
        print_SC = f"Last update SC: {time_SC_str.strftime('%d %B %Y at %H:%M')}"

    return print_SF, print_CM, print_SC

# Slider
@app.callback(
    Output('rangeSlider_hour_minute', 'max'),
    Output('rangeSlider_hour_minute', 'marks'),
    Output('rangeSlider_hour_minute', 'value'),
    Input('dropdown_day_month_year', 'value'),
    Input('interval_update', 'n_intervals'))
def update_slider(day_month_year, n):
    ctx = dash.callback_context
    triggering_id = ctx.triggered[0]['prop_id'].split('.')[0]
    
    # Initialize values
    now = pd.Timestamp.now(tz=constants.TIMEZONE)
    unique_times_days = unique_times[unique_times.map(lambda x: x.strftime('%d %B %Y')) == day_month_year]
    slider_marks = {i: {'label': time.strftime('%H:%M')} for i, time in enumerate(unique_times_days)}

    # Get indices for 08:00 and 17:00
    eight_oclock_index = unique_times_days.searchsorted(pd.Timestamp(day_month_year + " 8:00").tz_localize(constants.TIMEZONE))
    seventeen_oclock_index = unique_times_days.searchsorted(pd.Timestamp(day_month_year + " 17:00").tz_localize(constants.TIMEZONE))

    # Check if 'now' is present in the unique_times_days
    now_index = unique_times_days.searchsorted(now)
    is_now_present = now_index < len(unique_times_days) and now.day == pd.to_datetime(day_month_year).day

    # Determine min_show_value and max_show_value based on described conditions
    if is_now_present:
        min_show_value = now_index
        if now.hour >= 17:
            max_show_value = len(unique_times_days) - 1
        else:
            max_show_value = min(seventeen_oclock_index, len(unique_times_days) - 1)
        slider_marks[now_index]['style'] = {'color': 'red'}
    else:
        min_show_value = max(eight_oclock_index, 0)
        max_show_value = min(seventeen_oclock_index, len(unique_times_days) - 1)

    value = [min_show_value, max_show_value]
    max_value = len(unique_times_days) - 1

    # Update only with specific name
    if triggering_id == 'interval_update':
        return max_value, slider_marks, dash.no_update
    else:
        return max_value, slider_marks, value

# Axis callback
@app.callback(
    [Output('plot_velocity', 'figure', allow_duplicate=True),
     Output('plot_soc', 'figure', allow_duplicate=True),
     Output('plot_altitude', 'figure', allow_duplicate=True),
     Output('plot_inclination', 'figure', allow_duplicate=True),
     Output('plot_globalIrradiance', 'figure', allow_duplicate=True),
     Output('plot_temperature', 'figure', allow_duplicate=True),
     Output('plot_wind', 'figure', allow_duplicate=True),
     Output('plot_precipitation', 'figure', allow_duplicate=True)],
    [Input('plot_velocity', 'relayoutData'),
     Input('plot_soc', 'relayoutData'),
     Input('plot_altitude', 'relayoutData'),
     Input('plot_inclination', 'relayoutData'),
     Input('plot_globalIrradiance', 'relayoutData'),
     Input('plot_temperature', 'relayoutData'),
     Input('plot_wind', 'relayoutData'),
     Input('plot_precipitation', 'relayoutData')],
    [State('plot_velocity', 'figure'),
     State('plot_soc', 'figure'),
     State('plot_altitude', 'figure'),
     State('plot_inclination', 'figure'),
     State('plot_globalIrradiance', 'figure'),
     State('plot_temperature', 'figure'),
     State('plot_wind', 'figure'),
     State('plot_precipitation', 'figure')],
    prevent_initial_call=True)
def update_xaxis(v, soc, h, dh, gh, T, w, rr, fig_v, fig_soc, fig_h, fig_dh, fig_gh, fig_T, fig_w, fig_rr):
    # Get the callback context to determine which input triggered the callback
    ctx = dash.callback_context
    if not ctx.triggered:
        raise dash.exceptions.PreventUpdate
    input_id = ctx.triggered[0]['prop_id'].split('.')[0]

    trigger_value = ctx.triggered[0]['value']

    # PreventUpdate when autosize
    if 'autosize' in trigger_value and trigger_value['autosize']:
        if trigger_value['autosize']:
            raise dash.exceptions.PreventUpdate
    
    # PreventUpdate while dragging
    elif 'dragmode' in trigger_value and trigger_value['dragmode'] == 'pan':
            raise dash.exceptions.PreventUpdate
    
    # PreventUpdate when changing yxis only
    elif ('yaxis.range[0]' in trigger_value or 'yaxis.range[1]' in trigger_value) and not ('xaxis.range[0]' in trigger_value or 'xaxis.range[1]' in trigger_value):
        raise dash.exceptions.PreventUpdate
    
    # If autorange is set for any graph, let all graphs determine their x-axis range automatically
    elif 'xaxis.autorange' in trigger_value and trigger_value['xaxis.autorange']:
        fig_v['layout']['xaxis']['autorange'] = True
        fig_soc['layout']['xaxis']['autorange'] = True
        fig_h['layout']['xaxis']['autorange'] = True
        fig_dh['layout']['xaxis']['autorange'] = True
        fig_gh['layout']['xaxis']['autorange'] = True
        fig_T['layout']['xaxis']['autorange'] = True
        fig_w['layout']['xaxis']['autorange'] = True
        fig_rr['layout']['xaxis']['autorange'] = True

    else:
        # Determine the new x-axis range
        if input_id == 'plot_velocity':
            xaxis_range = [v['xaxis.range[0]'], v['xaxis.range[1]']]
        elif input_id == 'plot_soc':
            xaxis_range = [soc['xaxis.range[0]'], soc['xaxis.range[1]']]
        elif input_id == 'plot_altitude':
            xaxis_range = [h['xaxis.range[0]'], h['xaxis.range[1]']]
        elif input_id == 'plot_inclination':
            xaxis_range = [dh['xaxis.range[0]'], dh['xaxis.range[1]']]
        elif input_id == 'plot_globalIrradiance':
            xaxis_range = [gh['xaxis.range[0]'], gh['xaxis.range[1]']]
        elif input_id == 'plot_temperature':
            xaxis_range = [T['xaxis.range[0]'], T['xaxis.range[1]']]
        elif input_id == 'plot_wind':
            xaxis_range = [w['xaxis.range[0]'], w['xaxis.range[1]']]
        elif input_id == 'plot_precipitation':
            xaxis_range = [rr['xaxis.range[0]'], rr['xaxis.range[1]']]
        else:
            raise dash.exceptions.PreventUpdate

        # Update the x-axis range for all figures and set autorange to False
        fig_v['layout']['xaxis']['range'] = xaxis_range
        fig_v['layout']['xaxis']['autorange'] = False

        fig_soc['layout']['xaxis']['range'] = xaxis_range
        fig_soc['layout']['xaxis']['autorange'] = False

        fig_h['layout']['xaxis']['range'] = xaxis_range
        fig_h['layout']['xaxis']['autorange'] = False

        fig_dh['layout']['xaxis']['range'] = xaxis_range
        fig_dh['layout']['xaxis']['autorange'] = False

        fig_gh['layout']['xaxis']['range'] = xaxis_range
        fig_gh['layout']['xaxis']['autorange'] = False

        fig_T['layout']['xaxis']['range'] = xaxis_range
        fig_T['layout']['xaxis']['autorange'] = False

        fig_w['layout']['xaxis']['range'] = xaxis_range
        fig_w['layout']['xaxis']['autorange'] = False

        fig_rr['layout']['xaxis']['range'] = xaxis_range
        fig_rr['layout']['xaxis']['autorange'] = False

    return fig_v, fig_soc, fig_h, fig_dh, fig_gh, fig_T, fig_w, fig_rr

# Weather plots
@app.callback(
    Output('plot_globalIrradiance', 'figure'),
    Output('plot_temperature', 'figure'),
    Output('plot_wind', 'figure'),
    Output('plot_precipitation', 'figure'),
    Input('dropdown_day_month_year', 'value'),
    Input('rangeSlider_hour_minute', 'value'),
    Input('rangeSlider_hour_minute', 'marks'),
    Input('store_current_position', 'data'),
    State('plot_globalIrradiance', 'figure'),
    State('plot_temperature', 'figure'),
    State('plot_wind', 'figure'),
    State('plot_precipitation', 'figure'))
def update_graphs(day_month_year, hour_minute, slider_marks, current_position, fig_gh, fig_tt, fig_ww, fig_rr):

    fig_gh['layout']['shapes'][lenght_cs]['x0'] = current_position['cumDistance_km']
    fig_gh['layout']['shapes'][lenght_cs]['x1'] = current_position['cumDistance_km']

    fig_tt['layout']['shapes'][lenght_cs]['x0'] = current_position['cumDistance_km']
    fig_tt['layout']['shapes'][lenght_cs]['x1'] = current_position['cumDistance_km']

    fig_ww['layout']['shapes'][lenght_cs]['x0'] = current_position['cumDistance_km']
    fig_ww['layout']['shapes'][lenght_cs]['x1'] = current_position['cumDistance_km']

    fig_rr['layout']['shapes'][lenght_cs]['x0'] = current_position['cumDistance_km']
    fig_rr['layout']['shapes'][lenght_cs]['x1'] = current_position['cumDistance_km']

    # Extract timezone from the DataFrame's index (should be constants.TIMEZONE)
    timezone = preprocessed_df.index.get_level_values('time').tz 

    # Create the start and end timestamps for the day
    start_time = pd.Timestamp(day_month_year).tz_localize(timezone)
    end_time = start_time + pd.Timedelta(days=1)

    # Filter the DataFrame for the specific day
    data_for_selected_day = preprocessed_df[(preprocessed_df.index.get_level_values('time') >= start_time) & 
                            (preprocessed_df.index.get_level_values('time') < end_time)]
    
    # Create the start and end timestamps for the range
    first_datetime_str = day_month_year + " " + slider_marks[f'{hour_minute[0]}']['label'] # Slider marks are str and hour_minute int
    first_datetime_obj = pd.Timestamp(first_datetime_str).tz_localize(timezone)
    last_datetime_str = day_month_year + " " + slider_marks[f'{hour_minute[1]}']['label'] # Slider marks are str and hour_minute int
    last_datetime_obj = pd.Timestamp(last_datetime_str).tz_localize(timezone)

    # Create a mask to select the data in the range
    mask = (data_for_selected_day.index.get_level_values('time') >= first_datetime_obj) & (data_for_selected_day.index.get_level_values('time') <= last_datetime_obj)
    data_in_range_df = data_for_selected_day[mask]

    x_cumDistance_range_km = data_in_range_df.index.get_level_values('cumDistance').unique() / 1000

    fig_gh['data'] = [{
        'x': x_cumDistance_range_km,
        'y': data_in_range_df['globalIrradiance'].groupby(level='cumDistance').mean(),
        'type': 'scatter',
        'mode': 'lines',
        'hoverinfo': 'none'
    }]

    fig_tt['data'] = [{
        'x': x_cumDistance_range_km,
        'y': data_in_range_df['temperature'].groupby(level='cumDistance').mean(),
        'type': 'scatter',
        'mode': 'lines',
        'hoverinfo': 'none'
    }]

    # Combine series into a single dataframe
    combined_df = pd.concat({
        'Wind speed': data_in_range_df['windSpeed'].groupby(level='cumDistance').mean(),
        'Side wind': abs(data_in_range_df['sideWind'].groupby(level='cumDistance').mean()),
        'Front wind': data_in_range_df['frontWind'].groupby(level='cumDistance').mean()
    }, axis=1)

    # Replace wind_traces
    wind_traces = []
    for column in combined_df.columns:
        trace = {
            'x': x_cumDistance_range_km,
            'y': combined_df[column],
            'type': 'scatter',
            'mode': 'lines',
            'name': column,
            'hoverinfo': 'none'
        }
        wind_traces.append(trace)

    fig_ww['data'] = wind_traces

    fig_rr['data'] = [{
        'x': x_cumDistance_range_km,
        'y': data_in_range_df['precipitation'].groupby(level='cumDistance').mean(),
        'type': 'scatter',
        'mode': 'lines',
        'hoverinfo': 'none'
    }]

    return fig_gh, fig_tt, fig_ww, fig_rr

## CURRENT POSITION
# Store GPS data
@app.callback(
    Output('store_current_position', 'data'),
    Input('interval_update', 'n_intervals'))
def update_current_position(n):
    
    # random_number = random.randint(5000, 10000)
    # current_position = {
    #     'latitude': route_df['latitude'][random_number],
    #     'longitude': route_df['longitude'][random_number]
    # }
    # current_position = gps.get_current_location()
    current_cumDistance = route.find_closest_row(current_position)[0]['cumDistance']
    current_position['cumDistance'] = current_cumDistance
    current_position['cumDistance_km'] = current_cumDistance / 1000

    if current_position is None:
        raise dash.exceptions.PreventUpdate
    
    return current_position

# Show data in box and lines in map and weather plots
@app.callback(
    Output('current_lat', 'children'),
    Output('current_lng', 'children'),
    Output('dist_darw', 'children'),
    Output('dist_ade', 'children'),
    Output('map', 'figure'),
    Output('plot_velocity', 'figure'),
    Output('plot_soc', 'figure'),
    Output('plot_altitude', 'figure'),
    Output('plot_inclination', 'figure'),
    Input('store_current_position', 'data'),
    State('map', 'figure'),
    State('plot_velocity', 'figure'),
    State('plot_soc', 'figure'),
    State('plot_altitude', 'figure'),
    State('plot_inclination', 'figure'))
def plots_current_position(current_position, fig_map, fig_v, fig_soc, fig_h, fig_dh):
    ctx = dash.callback_context
    triggering_id = ctx.triggered[0]['prop_id'].split('.')[0]

    # Update vertical lines for all figures if current_position has changed
    if triggering_id == 'store_current_position':
        fig_map['data'][3]['lat'] = [current_position['latitude']]
        fig_map['data'][3]['lon'] = [current_position['longitude']]

        fig_v['layout']['shapes'][lenght_cs]['x0'] = current_position['cumDistance_km']
        fig_v['layout']['shapes'][lenght_cs]['x1'] = current_position['cumDistance_km']

        fig_soc['layout']['shapes'][lenght_cs]['x0'] = current_position['cumDistance_km']
        fig_soc['layout']['shapes'][lenght_cs]['x1'] = current_position['cumDistance_km']

        fig_h['layout']['shapes'][lenght_cs]['x0'] = current_position['cumDistance_km']
        fig_h['layout']['shapes'][lenght_cs]['x1'] = current_position['cumDistance_km']

        fig_dh['layout']['shapes'][lenght_cs]['x0'] = current_position['cumDistance_km']
        fig_dh['layout']['shapes'][lenght_cs]['x1'] = current_position['cumDistance_km']

        return current_position['latitude'], current_position['longitude'], round(current_position['cumDistance_km'], 1), round(max_cumDistance_km - current_position['cumDistance_km'], 1), fig_map, fig_v, fig_soc, fig_h, fig_dh
    
    else:
        raise dash.exceptions.PreventUpdate

## CONTROL STOP
@app.callback(
    [Output('cs_lat', 'children'),
     Output('cs_lng', 'children'),
     Output('cs_dist_dar', 'children'),
     Output('cs_dist_us', 'children'),
     Output('cs_name', 'children')],
    Input('store_current_position', 'data'))
def next_control_stop(current_position):
    row_route, _ = route.find_closest_row(current_position)
    current_cum_distance_km = row_route['cumDistance'] / 1000

    row, _ = route.find_next_cs(current_position, print_is_requested=False)

    if row is None:
        return "-", "-", "-", "-", "-"
    
    cs_cum_distance_km = row['cumDistance'] / 1000
    name = row['town'] + ': ' + row['location']

    delta_km = cs_cum_distance_km - current_cum_distance_km

    return row['latitude'], row['longitude'], round(cs_cum_distance_km, 1), round(delta_km, 1), name

# Submit button velocity
@app.callback(
    Output('submit_cs_v_time', 'disabled'),
    Input('cs_input_v_time', 'value'))
def update_button_cumDistance(input_velocity):
    if input_velocity is not None and 50 <= input_velocity <= 130:
        return False
    return True

    Input('submit_cum_dist', 'n_clicks'),
    State('input_cum_dist', 'value'),

# Velocity
@app.callback(
    Output('cs_max_v_time', 'children'),
    Output('cs_output_velocity', 'children'),
    # Output('cs_opt_v_time', 'children'),
    # Output('cs_mean_v_time', 'children'),
    Input('submit_cs_v_time', 'n_clicks'),
    State('cs_input_v_time', 'value'),
    State('store_current_position', 'data'))
def next_control_stop(n_clicks, input_speed, current_position):
    time_maxSpeed = time_space_forecaster.get_time_at_next_control_stop(current_position, type="max_speed")
    if n_clicks:
        output_maxSpeed = time_maxSpeed.strftime("%H:%M")
        
        if input_speed is None:
            return output_maxSpeed, dash.no_update
        
        time_inputSpeed = time_space_forecaster.get_time_at_next_control_stop(current_position, type="", speed=input_speed)
        output_inputSpeed = time_inputSpeed.strftime("%H:%M")

        return output_maxSpeed, output_inputSpeed
    else:
        raise dash.exceptions.PreventUpdate

## DRIVER STOP
# Dropdown time driver stop
@app.callback(
    [Output('dropdown_ds_hour', 'options'),
     Output('dropdown_ds_minute', 'options'),
     Output('dropdown_ds_hour', 'value'),
     Output('dropdown_ds_minute', 'value')],
    Input('interval_update_ds_time', 'n_intervals')
)
def update_dropdown_options(n):
    # Get the current time
    now = pd.Timestamp.now()
    current_hour = now.hour
    current_minute = now.minute

    hour_options = [{'label': str(i), 'value': i} for i in range(current_hour, 18)]
    if current_hour == now.hour:
        minute_options = [{'label': str(i), 'value': i} for i in range(current_minute, 60)]
    else:
        minute_options = [{'label': str(i), 'value': i} for i in range(60)]

    return hour_options, minute_options, current_hour, current_minute

# Submit button driver stop time
@app.callback(
    Output('submit_time_ds', 'disabled'),
    Input('dropdown_ds_hour', 'value')
)
def enable_button(hour):
    return hour >= 17  # If time is 17:00 or later, button is disabled

# Output time driver stop
@app.callback(
    Output('output_time', 'children'),
    Output('store_driver_stop', 'data'),
    [Input('submit_time_ds', 'n_clicks')],
    [State('dropdown_ds_hour', 'value'),
     State('dropdown_ds_minute', 'value')]
)
def display_time(n_clicks, hour, minute):
    if n_clicks is None:
        return dash.no_update, dash.no_update
    return f"Next driver stop: {hour:02}:{minute:02}", {'hour': hour, 'minute': minute}

# Driver stop
@app.callback(
    Output('ds_max_v_lat', 'children'),
    Output('ds_max_v_lng', 'children'),
    Output('ds_max_v_dist_darw', 'children'),
    Output('ds_max_v_dist_us', 'children'),
    Input('store_current_position', 'data'),
    Input('store_driver_stop', 'data'))
def next_overnight_stop(current_position, time):
    # PreventUpdate if time is an empty dict
    if not time:
        raise dash.exceptions.PreventUpdate
    
    os_cumDistance = time_space_forecaster.get_cum_distance_forecast(current_position, type="max_speed", time={'hour': 17, 'minute': 0})
    # os_cumDistance = time_space_forecaster.get_cum_distance_forecast(current_position, type="max_speed", time=time)
    
    # PreventUpdate if we are already in the overnight stop
    if os_cumDistance is None:
        raise dash.exceptions.PreventUpdate
    
    row, _ = route.find_closest_row_cumDistance(os_cumDistance)
    os_cum_distance_km = os_cumDistance / 1000

    delta_km = os_cum_distance_km - current_position['cumDistance_km']

    return row['latitude'], row['longitude'], round(os_cum_distance_km, 1), round(delta_km, 1)

## OVERNIGHT STOP
# Overnight stops
@app.callback(
    Output('os_max_v_lat', 'children'),
    Output('os_max_v_lng', 'children'),
    Output('os_max_v_dist_darw', 'children'),
    Output('os_max_v_dist_us', 'children'),
    Input('store_current_position', 'data'))
def next_overnight_stop(current_position):
    os_cumDistance = time_space_forecaster.get_cum_distance_forecast(current_position, type="max_speed", time={'hour': 17, 'minute': 0})

    # PreventUpdate if we are already in the overnight stop
    if os_cumDistance is None:
        raise dash.exceptions.PreventUpdate
    
    row, _ = route.find_closest_row_cumDistance(os_cumDistance)
    os_cum_distance_km = os_cumDistance / 1000
    # os_cum_dist_km2 = row['cumDistance'] / 1000
    # print(os_cum_dist_km, os_cum_dist_km2)

    delta_km = os_cum_distance_km - current_position['cumDistance_km']

    return row['latitude'], row['longitude'], round(os_cum_distance_km, 1), round(delta_km, 1)

# Submit button cumDistance
@app.callback(
    Output('submit_cum_dist', 'disabled'),
    Output('input_cum_dist', 'min'),
    Input('input_cum_dist', 'value'),
    State('store_current_position', 'data'))
def update_button_cumDistance(input_cumDistance_km, current_position):
    if input_cumDistance_km is not None and current_position['cumDistance_km'] <= input_cumDistance_km <= max_cumDistance_km:
        return False, current_position['cumDistance_km']
    return True, current_position['cumDistance_km']

# Output given a cumDistance
@app.callback(
    Output('os_output_velocity', 'children', allow_duplicate=True),
    Output('os_output_lat', 'children', allow_duplicate=True),
    Output('os_output_lng', 'children', allow_duplicate=True),
    Output('os_output_dist_darw', 'children', allow_duplicate=True),
    Output('os_output_dist_us', 'children', allow_duplicate=True),
    Input('submit_cum_dist', 'n_clicks'),
    State('input_cum_dist', 'value'),
    State('store_current_position', 'data'),
    prevent_initial_call=True)
def update_output_cumDistance(n_clicks, input_cumDistance_km, current_position):
    if n_clicks:
        # If smaller than current cumDistance
        if input_cumDistance_km < current_position['cumDistance_km']:
            raise dash.exceptions.PreventUpdate

        row, _ = route.find_closest_row_cumDistance(input_cumDistance_km * 1000)

        # Calculate the time difference between now and 17:00 of the current day
        now = pd.Timestamp.now()
        target_time = now.replace(hour=17, minute=0, second=0, microsecond=0)
        delta_hour = (target_time - now).seconds / 3600

        # Check if the current time is already past 17:00
        if delta_hour < 0:
            raise dash.exceptions.PreventUpdate
        
        mean_v = (input_cumDistance_km - current_position['cumDistance_km']) / delta_hour
        return round(mean_v, 1), row['latitude'], row['longitude'], round(input_cumDistance_km, 1), round(max_cumDistance_km - input_cumDistance_km, 1)
    
    else:
        raise dash.exceptions.PreventUpdate

# Submit button latitude and longitude
@app.callback(
    Output('submit_lat_lng', 'disabled'),
    Input('input_lat', 'value'),
    Input('input_lng', 'value'))
def update_button_latlon(input_lat, input_lng):
    if input_lat is not None and constants.GEO['latitude']['min'] <= input_lat <= constants.GEO['latitude']['max'] and input_lng is not None and constants.GEO['longitude']['min'] <= input_lng <= constants.GEO['longitude']['max']:
        return False
    return True

# Output given latitude and longitude
@app.callback(
    Output('os_output_velocity', 'children', allow_duplicate=True),
    Output('os_output_lat', 'children', allow_duplicate=True),
    Output('os_output_lng', 'children', allow_duplicate=True),
    Output('os_output_dist_darw', 'children', allow_duplicate=True),
    Output('os_output_dist_us', 'children', allow_duplicate=True),
    Input('submit_lat_lng', 'n_clicks'),
    State('input_lat', 'value'),
    State('input_lng', 'value'),
    State('store_current_position', 'data'),
    prevent_initial_call=True)
def update_output_latlon(n_clicks, input_lat, input_lng, current_position):
    if n_clicks:
        if input_lat is None or not (constants.GEO['latitude']['min'] <= input_lat <= constants.GEO['latitude']['max']) or input_lng is None or not (constants.GEO['longitude']['min'] <= input_lng <= constants.GEO['longitude']['max']):
            return 'Error: One or both values are out of range!'
        
        position = {'latitude': input_lat, 'longitude': input_lng}
        row, _ = route.find_closest_row(position)
        position_cumDistance_km = row['cumDistance'] / 1000

        # If smaller than current cumDistance
        if position_cumDistance_km <  current_position['cumDistance_km']:
            raise dash.exceptions.PreventUpdate
        
        # Calculate the time difference between now and 17:00 of the current day
        now = pd.Timestamp.now()
        target_time = now.replace(hour=17, minute=0, second=0, microsecond=0)
        delta_hour = (target_time - now).seconds / 3600

        # Check if the current time is already past 17:00
        if delta_hour < 0:
            raise dash.exceptions.PreventUpdate
        
        mean_v = (position_cumDistance_km -  current_position['cumDistance_km']) / delta_hour
        return round(mean_v, 1), input_lat, input_lng, round(position_cumDistance_km, 1), round(max_cumDistance_km - position_cumDistance_km, 1)
    else:
        raise dash.exceptions.PreventUpdate

# Camping table
@app.callback(
    Output('os_table', 'data'),
    Input('store_current_position', 'data'),
    Input('os_max_v_dist_darw', 'children'))
def update_table(current_position, os_max_v_dist):
    os_possible_camping_df = camping_cut_df[(camping_cut_df['cumDistance'] >= current_position['cumDistance']) & 
                                            (camping_cut_df['cumDistance'] <= os_max_v_dist * 1000)].copy()
    
    os_possible_camping_df['cumDistance_km'] = (os_possible_camping_df['cumDistance'] / 1000).round(1)

    return os_possible_camping_df.to_dict('records')

# Update map and output given camping location
@app.callback(
        Output('map', 'figure', allow_duplicate=True),
        Output('os_output_velocity', 'children'),
        Output('os_output_lat', 'children'),
        Output('os_output_lng', 'children'),
        Output('os_output_dist_darw', 'children'),
        Output('os_output_dist_us', 'children'),
        Input('os_table', 'active_cell'),
        State('map', 'figure'),
        State('os_table', 'data'),
        State('store_current_position', 'data'),
        prevent_initial_call=True)
def update_graphs(active_cell, fig_map, os_table, current_position):
    # Only update when the trigger comes from the table
    ctx = dash.callback_context
    triggering_id = ctx.triggered[0]['prop_id'].split('.')[0]
    
    # Update
    if triggering_id == 'os_table':
        table_row = active_cell['row']
        fig_map['data'][4]['lat'] = [os_table[table_row]['latitude']]
        fig_map['data'][4]['lon'] = [os_table[table_row]['longitude']]
        fig_map['data'][4]['hovertext'] = [os_table[table_row]['name']]

        input_cumDistance_km = os_table[table_row]['cumDistance_km']

        # If smaller than current cumDistance
        if input_cumDistance_km < current_position['cumDistance_km']:
            raise dash.exceptions.PreventUpdate

        # Calculate the time difference between now and 17:00 of the current day
        now = pd.Timestamp.now()
        target_time = now.replace(hour=17, minute=0, second=0, microsecond=0)
        delta_hour = (target_time - now).seconds / 3600

        # Check if the current time is already past 17:00
        if delta_hour < 0:
            raise dash.exceptions.PreventUpdate
        
        mean_v = (input_cumDistance_km - current_position['cumDistance_km']) / delta_hour

        return fig_map, round(mean_v, 1), os_table[table_row]['latitude'], os_table[table_row]['longitude'], round(input_cumDistance_km, 1), round(max_cumDistance_km - input_cumDistance_km, 1)
    else:
        raise dash.exceptions.PreventUpdate


if __name__ == '__main__':
    app.run_server(debug=True, mode="external", jupyter_mode="tab")
